In [12]:
import quantiacsToolbox
import pandas as pd
import numpy as np
import os
from os.path import dirname, abspath
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
import pickle

In [26]:
markets = ['CASH','F_AD','F_BO','F_BP','F_C','F_CC','F_CD','F_CL','F_CT','F_DX','F_EC',
                           'F_ED','F_ES','F_FC','F_FV','F_GC','F_HG','F_HO','F_JY','F_KC','F_LB','F_LC',
                           'F_LN','F_MD','F_MP','F_NG','F_NQ','F_NR','F_O','F_OJ','F_PA','F_PL','F_RB',
                           'F_RU','F_S','F_SB','F_SF','F_SI','F_SM','F_TU','F_TY','F_US','F_W','F_XX',
                           'F_YM','F_AX','F_CA','F_DT','F_UB','F_UZ','F_GS','F_LX','F_SS','F_DL','F_ZQ',
                           'F_VX','F_AE','F_BG','F_BC','F_LU','F_DM','F_AH','F_CF','F_DZ','F_FB','F_FL',
                           'F_FM','F_FP','F_FY','F_GX','F_HP','F_LR','F_LQ','F_ND','F_NY','F_PQ','F_RR',
                           'F_RF','F_RP','F_RY','F_SH','F_SX','F_TR','F_EB','F_VF','F_VT','F_VW','F_GD','F_F']

In [255]:
train_begin = '20180101'
train_end = '20201231'

In [256]:
data = quantiacsToolbox.loadData(markets, dataToLoad = ['DATE','OPEN','HIGH', 'LOW', 'CLOSE'], beginInSample = train_begin, endInSample = train_end)

Loading Data...
Done!


In [257]:
data['CLOSE'].shape

(776, 89)

In [258]:
date = np.array(list(map(lambda x : str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:], data['DATE'])))

In [259]:
df = pd.DataFrame(data['CLOSE'], columns = markets, index = date)

In [260]:
df.head()

,CASH,F_AD,F_BO,F_BP,F_C,F_CC,F_CD,F_CL,F_CT,F_DX,...,F_RY,F_SH,F_SX,F_TR,F_EB,F_VF,F_VT,F_VW,F_GD,F_F
2018-01-02,NaN,78300.0,20130.0,85156.25,17662.5,19360.0,80010.0,60370.0,38750.0,91573.0,...,16926250.0,25820.0,92690.0,39762.5,250812.5,128010.0,148800.0,112265.0,110675.0,251825.0
2018-01-03,NaN,78370.0,20340.0,84681.25,17650.0,19070.0,79805.0,61970.0,39055.0,91896.0,...,16913750.0,26130.0,93810.0,40037.5,250812.5,128090.0,149030.0,112270.0,111812.5,251825.0
2018-01-04,NaN,78640.0,20322.0,84887.50,17550.0,19050.0,80160.0,61890.0,39625.0,91581.0,...,17025000.0,26290.0,94290.0,40225.0,250812.5,128060.0,149110.0,112265.0,111850.0,251825.0
2018-01-05,NaN,78620.0,20256.0,84981.25,17562.5,18950.0,80630.0,61590.0,39005.0,91673.0,...,17052500.0,26520.0,94440.0,40187.5,250812.5,128080.0,149090.0,112265.0,110925.0,251825.0
2018-01-08,NaN,78390.0,20124.0,84987.50,17362.5,19140.0,80565.0,61900.0,39070.0,92100.0,...,16921250.0,26630.0,94470.0,39925.0,250812.5,128160.0,149300.0,112280.0,111012.5,251825.0


In [261]:
df.to_csv('data_'+train_begin+'_'+train_end+'.csv')

In [53]:
train_begin = '20180101'
train_end = '20201231'
df = pd.read_csv('data_'+train_begin+'_'+train_end+'.csv')
df = df.rename(columns = {'Unnamed: 0':'date'})
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [9]:
def tsplot(y, lags=None, figsize=(10, 8), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))    
        y.plot(ax=ts_ax)
        p_value = smt.adfuller(y)[1]
        ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)
        plt.tight_layout()
    return

In [53]:
_ = tsplot(df.loc['2020-01-01':,'F_AD'].dropna(), lags=10)
plt.show()
# There is no seasonality observed

In [20]:
'''
If we use loop to find best orders

best_order = {el:None for el in markets[1:]}
best_mdl = {el:None for el in markets[1:]}
best_aic = {el:np.inf for el in markets[1:]}
# orders = {el:[] for el in markets[1:]}
# aics = {el:[] for el in markets[1:]}

pq_rng = range(5) # [0,1,2,3,4]
d_rng = range(2) # [0,1]
for market in markets[1:]:
    for i in pq_rng:
        for d in d_rng:
            for j in pq_rng:
                try:
                    tmp_mdl = smt.ARIMA(df[market], order=(i,d,j)).fit(method='mle', trend='c')
                    tmp_aic = tmp_mdl.aic
#                     aics[market].append(tmp_aic)
#                     orders[market].append("("+str(i)+","+str(d)+","+str(j)+")")
                    if tmp_aic < best_aic[market]:
                        best_aic[market] = tmp_aic
                        best_order[market] = (i,d,j)
                        best_mdl[market] = tmp_mdl
                except: continue
'''

In [263]:
sarima_models = {el:None for el in markets[1:]}

In [264]:
for market in markets[1:]:
    # Train the model on train set (excluding the last 100 data points for validation)
    sarima_model = auto_arima(np.log(df[market]), trace=True, error_action='ignore', suppress_warnings=True)
    sarima_models[market] = sarima_model

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-5765.947, Time=0.34 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-5754.556, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-5763.141, Time=0.08 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-5761.511, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-5756.545, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-5766.022, Time=0.18 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-5766.264, Time=0.10 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-5768.415, Time=0.20 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-5760.464, Time=0.14 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=-5766.822, Time=0.22 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=-5765.477, Time=0.10 sec
 ARIMA(0,1,3)(0,0,0)[0]             : AIC=-5770.414, Time=0.10 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=-5768.257, Time=0.12 sec
 ARIMA(1,1,3)(0,0,0)[0]             : AIC=-5762.465, Time=0.13 sec
 ARIMA(0,1,4)(0,0,0

 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-6457.948, Time=0.11 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-6460.353, Time=0.13 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-6458.242, Time=0.28 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-6446.744, Time=0.10 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-6454.845, Time=0.21 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=-6462.333, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-6447.895, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-6460.526, Time=0.19 sec
 ARIMA(0,1,3)(0,0,0)[0]             : AIC=-6460.224, Time=0.16 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-6448.724, Time=0.05 sec
 ARIMA(1,1,3)(0,0,0)[0]             : AIC=-6457.588, Time=0.09 sec

Best model:  ARIMA(0,1,2)(0,0,0)[0]          
Total fit time: 1.803 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-6280.919, Time=0.16 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-6281.304, Time=0.07 sec
 ARIMA(1,

 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-3691.859, Time=0.27 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-3698.128, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-3696.198, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-3696.202, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-3699.848, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-3694.202, Time=0.09 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.579 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-6177.501, Time=0.25 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-6167.665, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-6173.480, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-6172.108, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-6169.255, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-6177.926, Time=0.08 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-6179.825, Time=0.33 sec
 ARIMA(0,

 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-4124.091, Time=0.07 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0]          
Total fit time: 1.397 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-4182.711, Time=0.13 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-4192.803, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-4191.112, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-4191.093, Time=0.15 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-4194.022, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-4190.262, Time=0.20 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.625 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-4072.177, Time=0.48 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-4068.286, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-4067.658, Time=0.09 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-4067.959, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]

 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-4100.782, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-4104.526, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-4102.808, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-4102.767, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-4106.526, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-4100.772, Time=0.05 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.311 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-6304.238, Time=0.24 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-6306.354, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-6304.382, Time=0.08 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-6304.355, Time=0.12 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-6307.737, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-6301.346, Time=0.12 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.639 seconds

...

/home/ai/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1890: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/home/ai/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1897: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-11622.074, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-11630.468, Time=0.09 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-11631.551, Time=0.37 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-11624.056, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-11631.196, Time=0.30 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-11630.898, Time=0.21 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-11615.935, Time=0.15 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-11633.429, Time=0.05 sec


/home/ai/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1890: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/home/ai/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1897: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-11632.978, Time=0.06 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=-11632.805, Time=0.14 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-11632.467, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-11618.013, Time=0.08 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 1.765 seconds


/home/ai/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1890: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/home/ai/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1897: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


In [265]:
sarima_models

{'F_AD': ARIMA(order=(0, 1, 3), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 'F_BO': ARIMA(order=(3, 2, 1), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 'F_BP': ARIMA(order=(0, 1, 3), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 'F_C': ARIMA(order=(0, 1, 0), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 'F_CC': ARIMA(order=(1, 1, 0), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 'F_CD': ARIMA(order=(0, 1, 0), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 'F_CL': ARIMA(order=(2, 1, 2), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 'F_CT': ARIMA(order=(0, 1, 0), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 'F_DX': ARIMA(order=(0, 1, 2), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 'F_EC': ARIMA(order=(0, 1, 0), scoring_args={}, suppress_warnings=True,
 

In [266]:
with open('sarima_models.pckl', 'wb') as f:
#     for market in sarima_models:
    pickle.dump(sarima_models, f)

In [14]:
with open('sarima_models.pckl', 'rb') as f:
#     while True:
#         try:
#             print(pickle.load(f))
#         except EOFError:
#             break
    sarima_models_new = pickle.load(f)

In [267]:
correct_dict = {el:None for el in markets[1:]}
for market in markets[1:]:
    correct = 0
    for i in range(len(df)-101, len(df)-1):
        model = sarima_models_new[market].fit(np.log(df[market][i-100:i+1]))
        fore = model.predict(1)[0]
        pos = 1 if fore > np.log(df[market][i]) else -1
        actual = 1 if df[market][i+1] > df[market][i] else -1
        if pos == actual:
            correct += 1
    correct_dict[market] = correct

In [268]:
correct_dict

{'F_AD': 50,
 'F_BO': 45,
 'F_BP': 56,
 'F_C': 37,
 'F_CC': 46,
 'F_CD': 41,
 'F_CL': 56,
 'F_CT': 47,
 'F_DX': 55,
 'F_EC': 50,
 'F_ED': 46,
 'F_ES': 43,
 'F_FC': 51,
 'F_FV': 42,
 'F_GC': 49,
 'F_HG': 41,
 'F_HO': 48,
 'F_JY': 57,
 'F_KC': 51,
 'F_LB': 61,
 'F_LC': 47,
 'F_LN': 46,
 'F_MD': 45,
 'F_MP': 43,
 'F_NG': 57,
 'F_NQ': 59,
 'F_NR': 43,
 'F_O': 52,
 'F_OJ': 45,
 'F_PA': 43,
 'F_PL': 54,
 'F_RB': 45,
 'F_RU': 50,
 'F_S': 36,
 'F_SB': 48,
 'F_SF': 44,
 'F_SI': 47,
 'F_SM': 50,
 'F_TU': 31,
 'F_TY': 52,
 'F_US': 48,
 'F_W': 46,
 'F_XX': 48,
 'F_YM': 48,
 'F_AX': 49,
 'F_CA': 45,
 'F_DT': 48,
 'F_UB': 51,
 'F_UZ': 52,
 'F_GS': 51,
 'F_LX': 58,
 'F_SS': 46,
 'F_DL': 48,
 'F_ZQ': 73,
 'F_VX': 51,
 'F_AE': 55,
 'F_BG': 46,
 'F_BC': 47,
 'F_LU': 60,
 'F_DM': 57,
 'F_AH': 47,
 'F_CF': 49,
 'F_DZ': 50,
 'F_FB': 50,
 'F_FL': 54,
 'F_FM': 41,
 'F_FP': 49,
 'F_FY': 45,
 'F_GX': 49,
 'F_HP': 54,
 'F_LR': 43,
 'F_LQ': 69,
 'F_ND': 51,
 'F_NY': 43,
 'F_PQ': 51,
 'F_RR': 50,
 'F_RF': 57,
 'F

In [272]:
correct_list = [0] + list(correct_dict.values())

In [273]:
correct_list = [max(0, x-50) for x in correct_list]

In [274]:
print(correct_list)

[0, 0, 0, 6, 0, 0, 0, 6, 0, 5, 0, 0, 0, 1, 0, 0, 0, 0, 7, 1, 11, 0, 0, 0, 0, 7, 9, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 8, 0, 0, 23, 1, 5, 0, 0, 10, 7, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 19, 1, 0, 1, 0, 7, 2, 4, 0, 1, 0, 18, 0, 0, 14, 0, 11]


In [275]:
with open('weights_list_sarima.txt', 'w') as f:
    for correct in correct_list:
        f.write(str(correct) + '\n')

# Use Information of Industry of Future

In [46]:
# Human defined
industry_dict = {'energy':['F_BC','F_BG','F_BO','F_CL','F_HO','F_NG','F_RB'],
                'metals':['F_GC','F_HG','F_PA','F_PL','F_SI'],
                'agriculture':['F_C','F_CC','F_CT','F_FC','F_KC','F_LB','F_LC','F_LN','F_NR','F_O','F_OJ','F_S','F_SB','F_SM','F_W'],
                'indices':['F_AE','F_AH','F_AX','F_CA','F_CF','F_DM','F_DX','F_FB','F_FP','F_FY','F_LX','F_MD','F_NQ','F_NY','F_RU','F_SX','F_VX','F_YM','F_XX'],
                'bond':['F_CF','F_EB','F_F','F_FV','F_GS','F_GX','F_SS','F_TU','F_TY','F_UB','F_US','F_ZQ'],
                'currency':['F_AD','F_BP','F_CD','F_ED','F_JY','F_LR','F_MP','F_ND','F_RR','F_SF','F_TR'],
                'others':[]}

In [47]:
for market in markets:
    flag = 0
    for key in industry_dict:
        if market in industry_dict[key]:
            flag = 1
    if flag == 0 and market != 'CASH':
        industry_dict['others'].append(market)

In [65]:
industry_dict_inv = {v:k for k in industry_dict for v in industry_dict[k]}

In [67]:
print(industry_dict_inv)

{'F_BC': 'energy', 'F_BG': 'energy', 'F_BO': 'energy', 'F_CL': 'energy', 'F_HO': 'energy', 'F_NG': 'energy', 'F_RB': 'energy', 'F_GC': 'metals', 'F_HG': 'metals', 'F_PA': 'metals', 'F_PL': 'metals', 'F_SI': 'metals', 'F_C': 'agriculture', 'F_CC': 'agriculture', 'F_CT': 'agriculture', 'F_FC': 'agriculture', 'F_KC': 'agriculture', 'F_LB': 'agriculture', 'F_LC': 'agriculture', 'F_LN': 'agriculture', 'F_NR': 'agriculture', 'F_O': 'agriculture', 'F_OJ': 'agriculture', 'F_S': 'agriculture', 'F_SB': 'agriculture', 'F_SM': 'agriculture', 'F_W': 'agriculture', 'F_AE': 'indices', 'F_AH': 'indices', 'F_AX': 'indices', 'F_CA': 'indices', 'F_CF': 'bond', 'F_DM': 'indices', 'F_DX': 'indices', 'F_FB': 'indices', 'F_FP': 'indices', 'F_FY': 'indices', 'F_LX': 'indices', 'F_MD': 'indices', 'F_NQ': 'indices', 'F_NY': 'indices', 'F_RU': 'indices', 'F_SX': 'indices', 'F_VX': 'indices', 'F_YM': 'indices', 'F_XX': 'indices', 'F_EB': 'bond', 'F_F': 'bond', 'F_FV': 'bond', 'F_GS': 'bond', 'F_GX': 'bond', 'F_SS

In [48]:
sum(len(industry_dict[key]) for key in industry_dict)

89

In [57]:
df_ind.head()

,F_EC,F_ES,F_DT,F_UZ,F_DL,F_LU,F_DZ,F_FL,F_FM,F_HP,...,F_PQ,F_RF,F_RP,F_RY,F_SH,F_VF,F_VT,F_VW,F_GD,avg
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,151375.00,134700.0,161280.0,111955.0,28080.0,95700.0,25500.0,55755.0,23700.0,30580.0,...,54640.0,146312.5,111037.50,16926250.0,25820.0,128010.0,148800.0,112265.0,110675.0,934186.7500
2018-01-03,150837.50,135462.5,161580.0,111975.0,28100.0,95050.0,25965.0,54810.0,23870.0,30190.0,...,55330.0,146637.5,111356.25,16913750.0,26130.0,128090.0,149030.0,112270.0,111812.5,933662.3125
2018-01-04,151493.75,136250.0,161590.0,111950.0,28080.0,94600.0,26195.0,53970.0,24060.0,28800.0,...,56170.0,146975.0,111531.25,17025000.0,26290.0,128060.0,149110.0,112265.0,111850.0,939424.5000
2018-01-05,150975.00,137125.0,161590.0,111955.0,27900.0,95000.0,26395.0,53550.0,24280.0,27950.0,...,56100.0,146812.5,111225.00,17052500.0,26520.0,128080.0,149090.0,112265.0,110925.0,940696.8750
2018-01-08,150193.75,137350.0,161800.0,111975.0,27700.0,95100.0,26700.0,52815.0,24400.0,28350.0,...,56400.0,146125.0,110450.00,16921250.0,26630.0,128160.0,149300.0,112280.0,111012.5,934172.0625


In [59]:
sarima_models_ind = {el:None for el in industry_dict}

In [60]:
for ind in industry_dict:
    df_ind = df[industry_dict[ind]]
    df_ind['avg'] = df_ind.mean(axis=1)
    sarima_model = auto_arima(np.log(df_ind['avg']), trace=True, error_action='ignore', suppress_warnings=True)
    sarima_models_ind[ind] = sarima_model

<ipython-input-60-79af906e81b7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind['avg'] = df_ind.mean(axis=1)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-3694.248, Time=0.84 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-3696.527, Time=0.09 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-3694.535, Time=0.10 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-3694.534, Time=0.15 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-3698.385, Time=0.06 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-3692.535, Time=0.25 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.536 seconds
Performing stepwise search to minimize aic


<ipython-input-60-79af906e81b7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind['avg'] = df_ind.mean(axis=1)


 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-4516.665, Time=0.24 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-4497.973, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-4507.169, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-4504.705, Time=0.12 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-4498.533, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-4518.375, Time=0.47 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-4520.618, Time=0.17 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-4518.686, Time=0.61 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-4510.663, Time=0.24 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-4516.658, Time=0.98 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=-4521.665, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-4505.485, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-4519.416, Time=0.16 sec
 ARIMA(0,1,3)(0,0,0)[0]             : AIC=-4519.739, Time=0.11 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-4511.883, Time=0.20

<ipython-input-60-79af906e81b7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind['avg'] = df_ind.mean(axis=1)


...

 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-7764.096, Time=0.51 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-7768.645, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-7769.516, Time=0.15 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-7769.333, Time=0.13 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-7768.583, Time=0.03 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-7768.330, Time=0.22 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-7767.151, Time=0.12 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-7766.082, Time=0.67 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-7769.690, Time=0.05 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=-7768.620, Time=0.11 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-7766.639, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-7769.487, Time=0.06 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-7762.579, Time=0.08 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0]          
Total fit time: 2.310 seconds
Performing stepwise search to minimize aic


<ipython-input-60-79af906e81b7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind['avg'] = df_ind.mean(axis=1)


 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-6996.826, Time=0.26 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-6986.594, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-6991.289, Time=0.16 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-6990.093, Time=0.08 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-6988.589, Time=0.04 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-6995.884, Time=0.21 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-6995.676, Time=0.20 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-6996.442, Time=0.35 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-6976.588, Time=0.36 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-6991.570, Time=0.14 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-6978.589, Time=0.33 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-6993.983, Time=0.18 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=-6974.588, Time=0.34 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=-6998.825, Time=0.12 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-6997.881, Time=0.14

<ipython-input-60-79af906e81b7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind['avg'] = df_ind.mean(axis=1)


 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-6193.488, Time=0.53 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-6199.089, Time=0.08 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-6198.662, Time=0.13 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-6198.570, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-6200.844, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-6195.185, Time=0.22 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.088 seconds


In [62]:
with open('sarima_models_ind.pckl', 'wb') as f:
    pickle.dump(sarima_models_ind, f)

In [69]:
correct_dict = {el:None for el in markets[1:]}
for market in markets[1:]:
    correct = 0
    print(market)
    for i in range(len(df)-101, len(df)-1):
        model = sarima_models_ind[industry_dict_inv[market]].fit(np.log(df[market][i-100:i+1]))
        fore = model.predict(1)[0]
        pos = 1 if fore > np.log(df[market][i]) else -1
        actual = 1 if df[market][i+1] > df[market][i] else -1
        if pos == actual:
            correct += 1
    correct_dict[market] = correct

F_AD
F_BO
F_BP
F_C
F_CC
F_CD
F_CL
F_CT
F_DX
F_EC
F_ED
F_ES
F_FC
F_FV
F_GC
F_HG
F_HO
F_JY
F_KC
F_LB
F_LC
F_LN
F_MD
F_MP
F_NG
F_NQ
F_NR
F_O
F_OJ
F_PA
F_PL
F_RB
F_RU
F_S
F_SB
F_SF
F_SI
F_SM
F_TU
F_TY
F_US
F_W
F_XX
F_YM
F_AX
F_CA
F_DT
F_UB
F_UZ
F_GS
F_LX
F_SS
F_DL
F_ZQ
F_VX
F_AE
F_BG
F_BC
F_LU
F_DM
F_AH
F_CF
F_DZ
F_FB
F_FL
F_FM
F_FP
F_FY
F_GX
F_HP
F_LR
F_LQ
F_ND
F_NY
F_PQ
F_RR
F_RF
F_RP
F_RY
F_SH
F_SX
F_TR
F_EB
F_VF
F_VT
F_VW
F_GD
F_F


In [71]:
correct_list = [0] + list(correct_dict.values())
correct_list = [max(0, x-50) for x in correct_list]
print(correct_list)
with open('weights_list_industry.txt', 'w') as f:
    for correct in correct_list:
        f.write(str(correct) + '\n')

[0, 0, 0, 0, 4, 1, 0, 0, 3, 5, 0, 0, 0, 2, 5, 0, 0, 0, 10, 4, 9, 7, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 6, 0, 0, 5, 0, 4, 3, 0, 6, 0, 0, 0, 0, 0, 3, 2, 0, 5, 0, 0, 12, 0, 5, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 3, 7, 2, 4, 0, 3, 0, 4, 0, 0, 0, 0, 10]
